<a href="https://colab.research.google.com/github/matheusvazdata/dados-lab/blob/main/Connecting_to_a_relational_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conectando ao Banco de Dados SQLite no Google Colab
Este script conecta-se a um banco de dados SQLite armazenado no Google Drive, lista suas tabelas e carrega uma tabela específica em um DataFrame Pandas para análise.

In [1]:
# Importação das bibliotecas necessárias
from sqlalchemy import create_engine, text, inspect
import pandas as pd
import os

In [2]:
# Caminho do arquivo
caminho_db = '/content/drive/MyDrive/dbs_sqlite/Chinook.sqlite'

In [3]:
# "Montagem" do Google Drive, na pasta content, para acessar arquivos armazenados nele
from google.colab import drive
print('Montando o Google Drive...')
drive.mount('/content/drive')

Montando o Google Drive...
Mounted at /content/drive


In [4]:
# Definição do caminho do banco de dados SQLite
caminho_db = '/content/drive/MyDrive/dbs_sqlite/Chinook.sqlite'

In [5]:
# Criação da URL de conexão para SQLite
url_conexao = f'sqlite:///{caminho_db}'

In [6]:
# Inicialização do SQLAlchemy Engine
print('Conectando ao banco de dados...')
engine = create_engine(url_conexao)
print('✅ Conexão bem-sucedida!')

Conectando ao banco de dados...
✅ Conexão bem-sucedida!


In [7]:
# Inspeção e listagem de todas as tabelas do banco de dados
inspetor = inspect(engine)
tabelas = inspetor.get_table_names()
print('\n📌 Tabelas disponíveis no banco de dados:')
for tabela in tabelas:
    print(f'- {tabela}')


📌 Tabelas disponíveis no banco de dados:
- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track


In [8]:
# Escolha de uma tabela específica para análise
tabela_escolhida = 'Invoice'

# Carregamento a tabela selecionada em um DataFrame Pandas
df = pd.read_sql_query(f"SELECT * FROM {tabela_escolhida}", engine)

# Exibição das primeiras linhas do DataFrame
print('\n📊 Visualização dos primeiros registros da tabela selecionada:\n')
display(df.head())


📊 Visualização dos primeiros registros da tabela selecionada:



,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


## Análises mais aprofundadas

In [11]:
# Exploração da tabela InvoiceLine
pd.read_sql_query(
    '''
    SELECT
      *
    FROM InvoiceLine
    LIMIT 3
    ''', engine
)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1


In [12]:
# Exploração da tabela Track
pd.read_sql_query(
    '''
    SELECT
      *
    FROM Track
    LIMIT 3
    ''', engine
)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99


In [18]:
# Identificação das 3 músicas (Name, na tabela Track) com os maiores volumes de vendas (Total, na tabela Invoice)
df_top_3_tracks = pd.read_sql_query(
    '''
    SELECT
      T.Name,
      SUM(I.Total) AS BillingTotal
    FROM Invoice I
    INNER JOIN InvoiceLine IL
      USING(InvoiceId) -- USING só pode ser usado quando os nomes das colunas que se relacionam são os mesmos
    INNER JOIN Track T
      USING(TrackId)
    GROUP BY T.Name
    ORDER BY BillingTotal DESC
    LIMIT 3
    ''', engine
)

# Exibição do resultado
df_top_3_tracks

,Name,BillingTotal
0,The Trooper,49.50
1,Eruption,45.54
2,Walkabout,43.77
